In [29]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
import pandas as pd

In [33]:
url_base = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/mazowieckie/warszawa/warszawa/warszawa?distanceRadius=0&limit=72&ownerTypeSingleSelect=ALL&by=DEFAULT&direction=DESC&viewType=listing&page='
num_pages = 50

headers = {'User-Agent': 'Mozilla/5.0'}

all_data = []

for page_num in range(1, num_pages + 1):
    url = url_base + str(page_num)

    req = Request(url, headers=headers)

    try:
        response = urlopen(req, timeout=10)
        page = response.read()

        soup = BeautifulSoup(page, 'html.parser')

        for t, p, l, a in zip(
            soup.find_all('div', {'class': 'css-gg4vpm e1n06ry51'}),
            soup.find_all('div', {'class': 'e1jyrtvq0 css-1tjkj49 ei6hyam0'}),
            soup.find_all('p', {'class': 'css-19dkezj e1n06ry53'}),
            soup.find_all('div', {'class': 'css-70qvj9 enzg89n0'})
        ):
            data = {
                'Listing': t.text.strip(),
                'Price1': p.text.strip(),
                'Location': l.text.strip(),
                'Ads': a.text.strip(),
            }
            all_data.append(data)

        print(f'Scraping page {page_num}')

    except Exception as e:
        print(f'Error: {e}')

Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Scraping page 11
Scraping page 12
Scraping page 13
Scraping page 14
Scraping page 15
Scraping page 16
Scraping page 17
Scraping page 18
Scraping page 19
Scraping page 20
Scraping page 21
Scraping page 22
Scraping page 23
Scraping page 24
Scraping page 25
Scraping page 26
Scraping page 27
Scraping page 28
Scraping page 29
Scraping page 30
Scraping page 31
Scraping page 32
Scraping page 33
Scraping page 34
Scraping page 35
Scraping page 36
Scraping page 37
Scraping page 38
Scraping page 39
Scraping page 40
Scraping page 41
Scraping page 42
Scraping page 43
Scraping page 44
Scraping page 45
Scraping page 46
Scraping page 47
Scraping page 48
Scraping page 49
Scraping page 50


In [35]:
df = pd.DataFrame(all_data)

In [36]:
divide = df['Price1'].str.split('zł', 1, expand=True)
divide.columns = ['Price', 'SQMUP1']

divide2 = divide['SQMUP1'].str.split('zł/m²', expand=True)
divide2.columns = ['SQMUP', 'Room Info1']

divide3 = divide2['Room Info1'].str.replace('pokój|pokoje|pokoi', 'room', regex=True).str.split('room', expand=True)
divide3.columns = ['Room Info', 'Area']

divide3['Area'] = divide3['Area'].str.replace('m²', ' ')

In [37]:
divide_df = pd.concat([divide, divide2, divide3], axis=1)
divide_df.drop(columns=['SQMUP1', 'Room Info1'], inplace=True)
divide_df.head()

,Price,SQMUP,Room Info,Area
0,660 000,11 551,2,57.14
1,860 000,17 917,3,48
2,755 000,15 567,3,48.5
3,2 177 000,17 416,4,125
4,539 000,17 387,1,31


In [38]:
final_df = pd.concat([df, divide_df], axis=1)
final_df.drop(columns=['Price1'], inplace=True)
final_df = final_df[['Listing', 'Price', 'SQMUP', 'Room Info', 'Area', 'Location', 'Ads']]

final_df.head()

,Listing,Price,SQMUP,Room Info,Area,Location,Ads
0,Dwu pokojowe mieszkanie w Warszawie (Stare Wlo...,660 000,11 551,2,57.14,"ul. Zapustna, Stare Włochy, Włochy, Warszawa, ...",Oferta prywatna
1,3 pokoje 48m2 + ogródek 100m2,860 000,17 917,3,48,"Natolin, Ursynów, Warszawa, mazowieckie",Oferta prywatna
2,Piękne 3 pokojowe mieszanie na sprzedaż 755 000zł,755 000,15 567,3,48.5,"Szczęśliwice, Ochota, Warszawa, mazowieckie",Oferta prywatna
3,"Piękny, nowoczesny apartament",2 177 000,17 416,4,125,"Zawady, Wilanów, Warszawa, mazowieckie",Eurovilla Wilanów Sp. z o.o.Biuro nieruchomości
4,Słoneczna/Inwestycyjna Kawalerka na Saskiej Kępie,539 000,17 387,1,31,"Saska Kępa, Praga-Południe, Warszawa, mazowieckie",homfi sp. z o.o.Biuro nieruchomości


In [39]:
final_df.to_csv('desktop/scrape.csv')